In [74]:
from PIL import Image
import numpy as np                         
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import json
import matplotlib.pyplot as plt

In [ ]:
def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]

            # If the pixel is not white...(we have white background)
            if pixel != (255, 255, 255):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

In [ ]:
brain_image_mask = Image.open('./data/test_mask.bmp')
test_image = Image.open('./data/book.bmp')

In [ ]:
create_sub_masks(brain_image_mask)

In [ ]:
img = create_sub_masks(brain_image_mask)['(255, 0, 0)']
img.show()

In [119]:
def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    #     measure.find_contours
    #Uses the “marching squares” method to compute a the iso-valued contours of 
    #the input 2D array for a particular level value. 
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')
    bboxes = []
    segmentations = []
    polygons = []
    areas = []
    annotation = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        #shapely.geometry Polygon
        
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)
    # Combine the polygons to calculate the bounding box and area
    offset = 0
    steps = 0
    for i , polygon in enumerate(polygons):
        
        bbox = []
        area = []
        if not polygon.is_empty:
            bbox = polygon.bounds
            area = polygon.area
        annotation = {
        'segmentation': segmentations[i],
        'iscrowd': 0,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area}
        annotations.append(annotation)
        steps +=1
        annotation_id += 1
    return annotations , steps
        
        

In [120]:
mask_images = [brain_image_mask]
lesion_id = 1
category_ids = {
    1: {
        '(255, 0, 0)': lesion_id,
    }
}
is_crowd = 0
annotation_id = 1
image_id = 1

annotations = []
for mask_image in mask_images:
    sub_masks = create_sub_masks(mask_image)
    for color, sub_mask in sub_masks.items():
        category_id = category_ids[image_id][color]
        annotation, steps = create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd)
        annotations=annotations+annotation
        annotation_id += steps
    image_id += 1

In [121]:
print(json.dumps(annotations))

[{"segmentation": [276.0, 89.5, 280.5, 88.0, 280.5, 86.0, 278.5, 85.0, 282.0, 83.5, 281.5, 82.0, 283.5, 81.0, 278.5, 78.0, 278.0, 73.5, 268.0, 74.5, 267.5, 76.0, 270.0, 78.5, 274.5, 81.0, 273.5, 82.0, 275.5, 86.0, 277.5, 87.0, 274.5, 88.0, 276.0, 89.5], "iscrowd": 0, "image_id": 1, "category_id": 1, "id": 1, "bbox": [267.5, 73.5, 283.5, 89.5], "area": 114.0}, {"segmentation": [], "iscrowd": 0, "image_id": 1, "category_id": 1, "id": 2, "bbox": [], "area": []}, {"segmentation": [200.0, 88.5, 202.0, 82.5, 195.0, 82.5, 193.5, 84.0, 194.0, 86.5, 199.0, 85.5, 200.0, 88.5], "iscrowd": 0, "image_id": 1, "category_id": 1, "id": 3, "bbox": [193.5, 82.5, 202.0, 88.5], "area": 28.25}, {"segmentation": [232.0, 91.5, 233.0, 87.5, 235.0, 89.5, 237.5, 89.0, 231.0, 82.5, 229.5, 85.0, 230.5, 91.0, 232.0, 91.5], "iscrowd": 0, "image_id": 1, "category_id": 1, "id": 4, "bbox": [229.5, 82.5, 237.5, 91.5], "area": 31.5}, {"segmentation": [330.0, 112.5, 330.5, 105.0, 327.5, 103.0, 326.0, 99.5, 322.5, 102.0, 3

In [93]:
x = [1,2]
y = [3, 2]


In [99]:
x

[1, 2, [3, 2], [3, 2]]